### 이미지 사이즈 600 x 600으로 증가
### 메모리 부족으로 실행 불가

In [1]:
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, LeakyReLU
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import tensorflow as tf
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt

### 이미지 전처리

In [2]:

"./plant-pathology-2020-fgvc7/images/"

IMAGE_PATH = "./plant-pathology-2020-fgvc7/images/"
TEST_PATH = "./plant-pathology-2020-fgvc7/test.csv"
TRAIN_PATH = "./plant-pathology-2020-fgvc7/train.csv"
#SUB_PATH = "./plant-pathology-2020-fgvc7/sample_submission.csv"

IMAGE_SIZE = (600, 600)

test_data = pd.read_csv(TEST_PATH)
train_data = pd.read_csv(TRAIN_PATH)

def load_image(image_id):
    file_path = image_id + ".jpg"
    image = cv2.imread(IMAGE_PATH + file_path)
    
    # 이미지 크기 확인
    height, width, _ = image.shape
    
    # 이미지 크기가 (2048, 1365)일 경우 회전하여 크기 변경
    if height == 2048 and width == 1365:
        image = np.rot90(image)

    # 이미지를 224, 224로 줄이기
    image = cv2.resize(image, IMAGE_SIZE)

    return image

    #return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

images = train_data["image_id"][:].apply(load_image)



In [3]:
images = np.stack(images)
labels = train_data[["healthy", "multiple_diseases", "rust", "scab"]].values

In [4]:
from sklearn.model_selection import train_test_split
# 데이터를 균등하게 분할하여 검증 세트를 생성
train_images, val_images, train_labels, val_labels = train_test_split(images, labels, test_size=0.2, stratify=labels, shuffle=True)

In [5]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   rotation_range=5,
                                   shear_range=0.7,
                                   zoom_range=[0.9, 2.2],
                                   fill_mode='nearest'
                                   )

val_datagen = ImageDataGenerator(rescale=1./255)

In [6]:
BATCH_SIZE = 5
train_generator = train_datagen.flow(x=train_images, y=train_labels, batch_size=BATCH_SIZE, shuffle=True)
val_generator = train_datagen.flow(x=val_images, y=val_labels, batch_size=BATCH_SIZE, shuffle=True)

In [14]:

from tensorflow.keras.applications import EfficientNetB7

efmodel = EfficientNetB7(include_top=False, weights='imagenet', classes=4, input_shape= (IMAGE_SIZE[1], IMAGE_SIZE[0], 3))
efmodel.trainable = False

model = Sequential()
model.add(efmodel)
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
#model.add(Dense(16, activation='relu'))
#model.add(Dense(8, activation='relu'))
model.add(Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])



In [16]:
model = Sequential()
model.add(Conv2D(16, kernel_size=(3,3), input_shape=(IMAGE_SIZE[1], IMAGE_SIZE[0],3), activation='relu'))
model.add(MaxPooling2D(pool_size=(4,4)))
model.add(Conv2D(32, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(4,4)))
model.add(Conv2D(64, kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(4,4)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [13]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb7 (Functional)  (None, 19, 19, 2560)     64097687  
                                                                 
 flatten_1 (Flatten)         (None, 924160)            0         
                                                                 
 dropout_1 (Dropout)         (None, 924160)            0         
                                                                 
 dense_2 (Dense)             (None, 4)                 3696644   
                                                                 
Total params: 67,794,331
Trainable params: 67,483,604
Non-trainable params: 310,727
_________________________________________________________________


In [17]:
# 학습 및 검증
modelpath = './test.hdf5'
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=30)
history = model.fit(train_generator, epochs=10000, verbose=1, callbacks=[early_stopping_callback, checkpointer], validation_data=val_generator)

Epoch 1/10000
292/292 [==============================] - ETA: 0s - loss: 1.2604 - accuracy: 0.3345
Epoch 1: val_loss improved from inf to 1.24320, saving model to .\test.hdf5
292/292 [==============================] - 84s 282ms/step - loss: 1.2604 - accuracy: 0.3345 - val_loss: 1.2432 - val_accuracy: 0.3425
Epoch 2/10000
292/292 [==============================] - ETA: 0s - loss: 1.2388 - accuracy: 0.3516
Epoch 2: val_loss improved from 1.24320 to 1.21170, saving model to .\test.hdf5
292/292 [==============================] - 79s 272ms/step - loss: 1.2388 - accuracy: 0.3516 - val_loss: 1.2117 - val_accuracy: 0.3589
Epoch 3/10000
292/292 [==============================] - ETA: 0s - loss: 1.2181 - accuracy: 0.3956
Epoch 3: val_loss did not improve from 1.21170
292/292 [==============================] - 79s 270ms/step - loss: 1.2181 - accuracy: 0.3956 - val_loss: 1.2291 - val_accuracy: 0.3616
Epoch 4/10000
292/292 [==============================] - ETA: 0s - loss: 1.0649 - accuracy: 0.5488

KeyboardInterrupt: 

In [18]:
# 6 훈련 과정 시각화 (정확도)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

# 7 훈련 과정 시각화 (손실)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

NameError: name 'history' is not defined